<a href="https://colab.research.google.com/github/dipanjan1311/COMP551/blob/master/VGG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy   as np  
import cv2
from scipy import ndimage
import scipy.misc # to visualize only
from scipy.misc import imresize
from skimage.restoration import denoise_bilateral
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
!pip install -q keras
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils.np_utils import to_categorical
from google.colab import files

Using TensorFlow backend.


In [0]:
URL_ENDPOINT = "http://cs.mcgill.ca/~ksinha4/datasets/kaggle/"

X_train = np.loadtxt(URL_ENDPOINT+"train_x.csv", delimiter=",")
print("Training Set Loaded")
Y_train = np.loadtxt(URL_ENDPOINT+"train_y.csv", delimiter=",")
X_test = np.loadtxt(URL_ENDPOINT+"test_x.csv", delimiter=",")
print("Test Set Loaded")

X_train = X_train.reshape(-1, 64, 64) # reshape 
Y_train = Y_train.reshape(-1, 1) 
X_test = X_test.reshape(-1, 64, 64) # reshape

Training Set Loaded
Test Set Loaded


In [0]:
####################### PREPROCESSING IMAGE ARRAYS #####################################
X_train_processed = np.where(X_train>250, 1, 0)
X_test_processed = np.where(X_test>250, 1, 0)

In [0]:
################################### GET A LIST OF BOUNDING BOX AROUND EACH DIGIT #########################################
def get_bounding_boxes(x):
    BB = []
    for i in range(x.shape[0]):
        img = x[i].copy()

        # Threshold, find contours (connected parts of image with the same value)
        ret_t, thresh = cv2.threshold(img,254,255,0)
        im2, contours, hierarchy = cv2.findContours(np.uint8(thresh), 0, 2)

        # Find min area rectangle that will encompass the contour
        L = []
        for c in contours:
            rect = cv2.minAreaRect(c)
            pos, size, orient = rect
            area = max(size[0],size[1])**2
            # Discard bounding boxes that cannot possibly be a digit..
            if area > 49:
                L.append((area,[pos[0],pos[1],size[0],size[1],orient]))
        
        # Sort by area, from largest to smallest
        L.sort(key=lambda x: x[0], reverse=True)
        L = list(list(zip(*L))[1])
        BB.append(L)
    
    return BB

In [0]:
########################################## Return coordinates of a tight, non-oriented box around each digit ############################################
def get_coord(img, bounding_box, offset=0):
    _, img_t = cv2.threshold(img,254,255,0)
    x_pos, y_pos, width, height, orient = bounding_box
    box = cv2.boxPoints(((x_pos, y_pos),(width, height),orient))
    box = np.int0(box)

    x_min = max(min(box[:,0]),0)
    x_max = min(max(box[:,0]),img_t.shape[0])
    y_min = max(min(box[:,1]),0)
    y_max = min(max(box[:,1]),img_t.shape[0])
    
    # Original bounding box found (discarding orientation...)
    digit = img_t[y_min:y_max,x_min:x_max].copy()
    
    # Since we've discarded orientation info, we should tighten up the bounding box to compensate
    s_x = np.sum(digit,axis=0)
    x_nz = np.nonzero(s_x)
    x_min += np.amin(x_nz)
    x_max -= (digit.shape[1] - np.amax(x_nz))

    s_y = np.sum(digit,axis=1)
    y_nz = np.nonzero(s_y)
    y_min += np.amin(y_nz)
    y_max -= (digit.shape[0] - np.amax(y_nz))
    
    x_min = max(x_min-offset,0)
    x_max = min(x_max+offset,img_t.shape[0])
    y_min = max(y_min-offset,0)
    y_max = min(y_max+offset,img_t.shape[0])
    
    width = x_max-x_min
    height = y_max-y_min
    
    return x_min, y_min, width, height

In [0]:
############################### GET THE DIGIT FROM BOUNDING BOX #########################################
def get_digit(img, bounding_box, offset=0):
    x, y, w, h = get_coord(img, bounding_box)
    digit = digit[y:y+h,x:x+w].copy()
    return digit

In [0]:
################################################ GET ORIENTED BOUNDED BOX FOR ALL ENTRIES ###################################
# Oriented bounding boxes for all digits

def get_processed_dataset(dataset):
  TRAIN_BB = get_bounding_boxes(dataset)

  DISCARD_THRESH = 64
  TRAIN_X_ORIG = []
  TRAIN_X_PROC = []
  TRAIN_Y_PROC = []
  TRAIN_BB_PROC = []
  
  for i in range(dataset.shape[0]):

      # Cannot possibly be a single digit... Discard...
      x_orig, y_orig, w_orig, h_orig, _ = TRAIN_BB[i][0]
      if w_orig >= DISCARD_THRESH or h_orig >= DISCARD_THRESH:
        print(i)
        continue

      # Get un-rotated digit and threshold...
      x,y,w,h = get_coord(dataset[i], TRAIN_BB[i][0])
      x,y,w,h = int(x),int(y),int(w),int(h)
      _, img_t = cv2.threshold(dataset[i,y:y+h,x:x+w],254,255,0)  

      # Pad such that we have a square...
      max_wh = max(w,h)
      if max_wh > w:
        pad_amt = int((max_wh-w)/2)
        img_t = np.pad(img_t, ((0,0),(pad_amt,pad_amt)), 'constant', constant_values=0)
      elif max_wh > h:
        pad_amt = int((max_wh-h)/2)
        img_t = np.pad(img_t, ((pad_amt,pad_amt),(0,0)), 'constant', constant_values=0)

      # Pad such that we have a border of 2 pixels...
      img_t = np.pad(img_t, 2, 'constant', constant_values=0)
      img_t = cv2.resize(img_t, (28,28))

      TRAIN_X_ORIG.append(dataset[i])
      TRAIN_X_PROC.append(img_t)
      TRAIN_Y_PROC.append(Y_train[i])
      TRAIN_BB_PROC.append(TRAIN_BB[i])

  TRAIN_X_ORIG = np.array(TRAIN_X_ORIG)
  TRAIN_X_PROC = np.expand_dims(np.array(TRAIN_X_PROC),axis=1)
  TRAIN_Y_PROC = np.array(TRAIN_Y_PROC)
  TRAIN_BB_PROC = np.array(TRAIN_BB_PROC)

  # Normalize to be in range [-1,1]
  TRAIN_X_PROC = (TRAIN_X_PROC/255)*2-1
  
  return TRAIN_X_PROC, TRAIN_Y_PROC

In [0]:
########################### RETRIVE PROCESSED DATASET FROM ABOVE METHODS ##############################

X_train_processed, Y_train_processed = get_processed_dataset(X_train) 
X_test_processed, temp = get_processed_dataset(X_test)
del(temp)

In [0]:
############################## FLATENNING THE TRAINING AND TEST SET ####################################################
X_train_ready, X_test_ready = [],[]


#Training set
for image in X_train_processed:
    flat_img = image.reshape((784))
    X_train_ready.append(flat_img)
    
#Test set
for image in X_test_processed:
    flat_img = image.reshape((784))
    X_test_ready.append(flat_img)


X_train_ready = np.array(X_train_ready)
X_test_ready = np.array(X_test_ready)

In [0]:
x_train = X_train_ready
y_train = Y_train
x_valid = X_train_ready[37500:]
y_valid = Y_train[37500:]

x_test = X_test_ready
x_train = x_train.reshape(x_train.shape[0], 28, 28,1).astype('float32')
x_valid = x_valid.reshape(x_valid.shape[0], 28, 28,1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], 28, 28,1).astype('float32')
num_classes = 10

def larger_model():
  # create model
  model = Sequential()
  
  model.add(Conv2D(64, 3, 3, input_shape=(28, 28,1), activation='relu'))
  model.add(Conv2D(64, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Conv2D(128, 3, 3, activation='relu'))
  model.add(Conv2D(128, 3, 3, activation='relu'))
  model.add(MaxPooling2D((2, 2), strides=(2,2)))
  model.add(Dropout(0.25))
  
  model.add(Flatten())
  model.add(Dense(2000, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(2000, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(num_classes, activation='softmax'))
  # Compile model
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [0]:
model = larger_model()
# Fit the model
y_binary = to_categorical(y_train)
y_valid_bin = to_categorical(y_valid)
model.fit(x_train, y_binary, epochs=5, batch_size=100)

# Final evaluation of the model
scores = model.evaluate(x_valid, y_valid_bin, verbose=0)
print("Deep CNN Accuracy: %.2f%%" % (100-scores[1]*100))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(28, 28, 1..., activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`


Epoch 1/5
50000/50000 [==============================] - 426s 9ms/step - loss: 0.5151 - acc: 0.8555
Epoch 2/5
26800/50000 [===============>..............] - ETA: 3:18 - loss: 0.3093 - acc: 0.9235

50000/50000 [==============================] - 426s 9ms/step - loss: 0.3059 - acc: 0.9251
Epoch 3/5
43300/50000 [========================>.....] - ETA: 56s - loss: 0.2684 - acc: 0.9351

50000/50000 [==============================] - 422s 8ms/step - loss: 0.2674 - acc: 0.9354
Epoch 4/5
50000/50000 [==============================] - 422s 8ms/step - loss: 0.2415 - acc: 0.9411
Epoch 5/5
  500/50000 [..............................] - ETA: 7:01 - loss: 0.2014 - acc: 0.9400

50000/50000 [==============================] - 427s 9ms/step - loss: 0.2219 - acc: 0.9458
Deep CNN Accuracy: 3.78%


In [0]:
#Predict labels on test set
eval_results = model.predict(x_test, batch_size=None, verbose=0, steps=None)

In [0]:
cnn_pred = []

for pred in eval_results:
  cnn_pred.append(np.argmax(pred))
cnn_pred = np.array(cnn_pred, dtype = np.int32)

In [0]:
######################### SAVING PREDICTIONS TO FILE ############################################
index = np.arange(len(X_test))

#Formatting CNN data for submission
cnn_submission = []
cnn_submission.append('Id,Label')
for i in range(len(cnn_pred)):
    string = str(index[i]) + ',' + str(cnn_pred[i])
    cnn_submission.append(string)
cnn_submission = np.array(cnn_submission)

#Save in csv file for Convoluted Neural Nets
np.savetxt('DEEPCNN.csv', cnn_submission, fmt="%s")


files.download('DEEPCNN.csv')